In [129]:
import random
import pandas as pd
import numpy as np
from collections import Counter
from scipy import stats
import itertools
from joblib import Parallel, delayed

#Calculates one p val from bootstraps
def bootstrap_enrich_p(idx,row,var1,var2,bootstraped_enrichments):
    var1_obs = row[0]
    var2_obs = row[1]
    observed = row['observed_enrich']
    tmp_df = bootstraped_enrichments[bootstraped_enrichments.loc[:,var2] == var2_obs]
    tmp_df = tmp_df[tmp_df.loc[:,var1] == var1_obs]
    p = (100-stats.percentileofscore(tmp_df.loc[:,'bootstrapped_fold_expected'],observed))/100
    return(p)

#generates one bootstrap
def bootstrap_enrich(i,samped_var2_list,var2,var2_list,var1,var1_list,df,total_genes,expected_df):
    random.seed(i)
    tmp_var2_list = samped_var2_list
    random.shuffle(tmp_var2_list)
    tmp_dict = {'tmp': tmp_var2_list} 
    tmp_df = pd.DataFrame(tmp_dict)
    tmp_df = pd.concat([df,tmp_df],axis=1)
    tmp_df = tmp_df[[var1,'tmp']]
    
    holder2 = list()
    for var1_obs in var1_list:
        for var2_obs in var2_list:
            tmp_df2 = tmp_df[tmp_df.loc[:,var1]==var1_obs]
            tmp_df2 = tmp_df2[tmp_df2.loc[:,'tmp']==var2_obs]
            holder2.append(len(tmp_df2)/total_genes)
    expected_df["bootstrapped_fold_expected"] = holder2/expected_df['expected_freq']
    expected_df = expected_df[[0,1,'bootstrapped_fold_expected']]
    expected_df = expected_df.rename(columns={0:var1, 1:var2})
    return(expected_df)

#tests for significant enrichments in the intersection of two categorical variables based 
#using bootstrapping. These variables can have any number of labels, but labels must be exclusive 
#cpu parallelized using joblib's Parallel and delayed with the default Loky backend
def bootstrapped_enrichment_test(df,var1,var2,n_boots,n_jobs):
    
    #get the elements
    samped_var1_list = df.loc[:,var1].tolist()
    samped_var2_list = df.loc[:,var2].tolist()
    
    
    #count and frequency or elements
    var1_counted = Counter(samped_var1_list)
    var1_counted = pd.DataFrame.from_dict(var1_counted, orient='index').reset_index()\
        .rename(columns={'index':var1, 0:'count'})
    var1_counted["freq"] = var1_counted['count']/sum(var1_counted['count'])
    var1_counted = var1_counted.sort_values(by=var1)

    var2_counted=Counter(samped_var2_list)
    var2_counted = pd.DataFrame.from_dict(var2_counted, orient='index').reset_index()\
        .rename(columns={'index':var2, 0:'count'})
    var2_counted["freq"] = var2_counted['count']/sum(var2_counted['count'])
    var2_counted = var2_counted.sort_values(by=var2)
    
    
    #calculate the expected frequencies
    var1_list = var1_counted.loc[:,var1]
    var2_list = var2_counted.loc[:,var2]
    var1_freq = np.matrix(var1_counted.loc[:,'freq'])
    var2_freq  = np.matrix(var2_counted.loc[:,'freq'])
    tmp=var1_freq.T*var2_freq
    expected = tmp.ravel().tolist()
    expected = expected[0]
    expected_df = pd.DataFrame(itertools.product(var1_list, var2_list))
    expected_df['expected_freq'] = expected
    
    #calculate the observed frequency and fold-enrichment
    holder0 = list()
    total_genes = len(df)

    for idx, row in expected_df.iterrows():
        var1_obs = row[0]
        var2_obs = row[1]

        tmp_df = df[df.loc[:,var1] == var1_obs]
        tmp_df = tmp_df[tmp_df.loc[:,var2] == var2_obs]
        holder0.append(len(tmp_df)/total_genes)
    expected_df['observed_freq'] = holder0
    expected_df['observed_enrich'] = expected_df['observed_freq']/expected_df['expected_freq']

        
    #generate bootstraps and calculate fold enrichments
    bootstraped_enrichments = Parallel(n_jobs=n_jobs)(
        delayed(bootstrap_enrich)(i,samped_var2_list=samped_var2_list,var2=var2,var2_list=var2_list,\
                                  var1=var1,var1_list=var1_list,df=df,total_genes=total_genes,
                                  expected_df = expected_df)\
        for i in range(n_boots))

    bootstraped_enrichments = pd.concat(bootstraped_enrichments)
    
    #calculate unadjusted p values
    expected_df['p'] = Parallel(n_jobs=n_jobs)(
        delayed(bootstrap_enrich_p)(idx,row,var1,var2,bootstraped_enrichments) \
        for idx,row in expected_df.iterrows())
    expected_df = expected_df.rename(columns={0:var1, 1:var2})
    
    return(expected_df)

In [131]:
df = pd.read_csv('chr_convert.csv')
remapped_to_new_assembly = bootstrapped_enrichment_test(
    df=df,var1='new_chr',var2='me_assigned',n_boots=50000,n_jobs=30)

In [132]:
df = pd.read_csv('g_mem.csv')
old_assembly = bootstrapped_enrichment_test(
    df=df,var1='chr',var2='me_assigned',n_boots=50000,n_jobs=30)

0.3806611140031234
0.43445960098444786
0.966248212435455
0.990075784915193
1.1679864253393666
1.2592399791775117
1.3857384680490898
1.1038189701253192
1.784274193548387
0.2645898110499881
0.28740764796622553
0.4730870686037449
0.3126447429365447
1.0459212616292262
0.3372028043080674
1.178882244982535
0.1981988707313101
1.2021787575577318
0.45400136798905605
1.2366915091828588
0.8899839098954143
1.1666632603008502
0.608275293255132
0.41216467958271236
1.3268631057092595
0.30925313329916604
0.31244115985690085
0.5109385103303877
0.20526657595249878
1.5975498218927506
1.784274193548387
1.3070447496677005
0.8228985866600544
1.0214419451640206
1.1128529273690564
1.0858895705521472
1.0637019230769231
1.3590577824049976
0.7865002115954296
1.041470532856336
1.7779924996173275
0.31244115985690085
0.5109385103303877
0.17692922830867652
1.245777027027027
1.0307860655224257
1.3593213160908573
0.9549168559271151
1.2521866639226178
0.9632891351672275
1.1570954440309764
0.7914348171701112
1.537169986

1.1128529273690564
1.068561545490145
0.9901480666939445
1.041470532856336
1.7779924996173275
0.9789823008849557
0.07325350402825297
1.3024602026049203
0.26618854102612094
1.1608028996025725
1.4493940386505075
0.9870307819769295
0.24194430269009254
1.1298032596207037
1.201448764715636
0.7546533913721415
1.9117223502304144
0.32384367681498827
2.0021416505791505
1.3999226659167603
3.973975182270291
0.2893543746457998
1.0542779993646536
0.0
1.231190150478796
1.3163540488944083
0.4162224869881483
1.4551400332499043
0.9956824432178778
0.8669670846394985
0.7034976152623211
0.4099119962945808
0.9328588009569605
1.0883436664686665
0.5644582635733079
2.6672175772895192
0.2645898110499881
0.17307466656236553
0.37704498977505113
1.1584265608083724
1.0619393681091072
1.0666702861175963
0.9491768548351504
0.2159519781363873
0.6780454584149145
0.2976635374312366
1.0384479993741933
1.2348837209302326
0.7208144796380092
2.6047798445961856
1.0589924614095967
1.041470532856336
1.7779924996173275
0.289354

1.7715065785677189
1.0839389238180779
1.8947593786426244
1.1196862348178138
0.7949630713953024
0.386440381928272
1.3056806259835636
2.032371653114687
1.0732476352170748
0.5454209293726118
0.3372028043080674
0.1981988707313101
1.0890462262804965
0.8884749241477782
2.483908390090562
1.4551400332499043
0.8376874148114493
0.17028399907642575
1.2275640136488473
1.3172909947903746
2.4242147552958366
2.1188018727388807
1.1866452131938858
1.254087638979725
0.8611221056797941
1.115171370967742
1.0947759063659719
1.5917266187050356
1.4369993505087681
0.2893543746457998
0.2918694301125704
0.8228985866600544
0.5213595872681508
0.3527864147333174
1.3430434899908272
0.6968430777629632
1.0275474748864732
1.2440092104206273
0.4223134185913342
0.4292365893879135
1.5519334532374098
2.6672175772895192
0.0
0.8609942059900264
1.0625950532298087
0.386440381928272
0.7222288717452096
0.9016382308193871
1.2631094853469786
0.3030268444119796
1.0594009363694403
1.156568773369151
0.9283788973600365
1.186645213193

0.5187471913405365
0.5187471913405365
0.386440381928272
0.7222288717452096
0.9016382308193871
1.2631094853469786
1.3590577824049976
0.7865002115954296
0.2976635374312366
1.0384479993741933
0.8611221056797941
1.115171370967742
1.0947759063659719
1.5917266187050356
0.7015272419806584
0.8315370585309235
0.5644582635733079
0.28740764796622553
2.395367093946992
0.5213595872681508
0.9227869422238858
0.5626344845573692
1.2501558020690513
0.6501418583443104
0.828605709212972
0.7226816798152692
0.893979527700458
1.3915337444749212
1.1022543280607797
1.400405088929679
1.212107377647918
0.9416897212172802
0.3674809217417328
0.2918694301125704
3.714028776978417
0.5713076597545493
0.601348250314413
1.1305832346177969
1.3430434899908272
0.6968430777629632
0.608275293255132
0.41216467958271236
1.3268631057092595
0.30925313329916604
0.20526657595249878
1.5975498218927506
0.35158112188145557
1.7867237341516595
0.354037764028163
0.3771021134762518
1.0104431487768117
1.0459212616292262
0.1981988707313101

1.0104431487768117
0.9549168559271151
0.9272692103099761
0.9227869422238858
0.26618854102612094
1.1608028996025725
0.9981066560949271
1.18592242471494
1.1838645693989573
0.7898793746957321
0.7222288717452096
1.0858895705521472
1.0637019230769231
0.9491768548351504
1.0140848077662163
1.3430434899908272
0.6968430777629632
1.2501558020690513
1.052769318614389
0.49073971387379395
1.0064304027789266
0.6247898877159955
1.2594876660341556
1.0667791706846674
0.8899839098954143
1.1666632603008502
2.249428111497077
0.655345197837731
1.156568773369151
0.9283788973600365
0.17621681415929205
0.3842257597684515
1.2348837209302326
0.7208144796380092
0.43727096800757603
0.024905724095232736
0.0
1.0307860655224257
1.0104431487768117
0.8917742708575414
0.9877516633543593
1.1485983206933912
1.2275640136488473
1.3172909947903746
0.29666130329847146
0.5174757707319213
0.45889795353982304
0.6003527496382055
0.30148528343023256
1.8771210407239818
0.9416897212172802
1.0976385746606336
0.18000244071106047
0.27

0.5213595872681508
1.3037201445631679
2.2322520484285193
0.065556850505689
0.6003527496382055
0.5168319144518272
2.4134413380736914
0.8059336656441718
1.1218731219951925
1.590846238938053
1.193794964028777
0.9605643994211287
0.5644582635733079
0.12747263299404646
0.0
1.245777027027027
0.506268081002893
1.1128529273690564
0.9549168559271151
1.0858895705521472
1.0637019230769231
0.9644725370531823
0.49014178112538764
0.990075784915193
1.1679864253393666
1.2592399791775117
1.3857384680490898
0.3868044561427364
0.5702972849123611
0.07325350402825297
0.0
0.7115749525616698
1.4067648022196104
1.2521866639226178
0.7703946277565911
1.1542920631853535
1.2571022727272727
0.8763453343531469
1.0338053097345135
1.2988489208633094
0.17621681415929205
0.3842257597684515
0.0
0.3754242081447964
1.784274193548387
1.3601434426229508
0.0
0.4891520627293764
0.661102584649653
0.4162224869881483
1.4551400332499043
0.9956824432178778
0.8669670846394985
1.1866452131938858
1.254087638979725
0.22679126661427548


1.2833377048715477
1.0038900634249472
0.8479285714285714
1.362603305785124
0.6402360098373387
0.7391656288916564
1.063664825434306
0.0
0.8104669862847432
2.3981818181818184
0.6532577591899627
2.158363636363636
0.8975582636541539
0.0
1.8353432282003712
0.0
0.0
0.3806611140031234
0.29725251349112164
0.386440381928272
0.4162224869881483
0.2159519781363873
2.249428111497077
0.43445960098444786
2.0021416505791505
0.22679126661427548
1.9641448017360428
0.7884421215180852
1.3255558514179204
12.933344158480152
4.674707280570473
1.1403189980736093
0.6587702954484962
1.4067648022196104
0.8161051781072157
0.9981066560949271
1.1838645693989573
1.1196862348178138
1.0064304027789266
0.6247898877159955
0.19844235828749107
1.2331569992568545
0.17621681415929205
0.3842257597684515
1.2348837209302326
0.7208144796380092
0.9789823008849557
0.09177959070796461
0.17307466656236553
0.354037764028163
0.4891520627293764
4.674707280570473
1.233237417883707
0.20442785865347105
1.4709338831042809
1.20594113372093

In [134]:
old_assembly.to_csv('old_enrich.csv')
remapped_to_new_assembly.to_csv('remapped_enrich.csv')